In [ ]:
from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.credentials import load_account, get_token_authentication, get_basic_authentication
from quantuminspire.api import QuantumInspireAPI
from qiskit import IBMQ, QuantumRegister, ClassicalRegister, QuantumCircuit, assemble, transpile
from qiskit.providers.ibmq import least_busy
from qiskit.circuit import Parameter 



import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
theta_param = Parameter('theta_param')

qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(2, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.ry(theta_param, qreg_q[0])
circuit.sdg(qreg_q[1])
circuit.h(qreg_q[1])
circuit.tdg(qreg_q[1])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.t(qreg_q[1])
circuit.h(qreg_q[1])
circuit.s(qreg_q[1])
circuit.cx(qreg_q[1], qreg_q[0])
circuit.ry(-theta_param, qreg_q[1])
circuit.measure(qreg_q[1], creg_c[1])

In [ ]:
IBMQ.save_account('')

N_shots=1024
provider = IBMQ.load_account()

#backend = least_busy(provider.backends(simulator=False))
#print(backend)
backend = provider.backends.ibmq_santiago

In [ ]:
N_shots=1024
N=100
runningjobs=[]
job_ids=[]
index=0
points=(np.linspace(0,np.pi,N),np.zeros(N))

while index*75 < num_pts:
    if((index+1)*75<num_pts):
        circuits = [circuit.bind_parameters(
        {theta_param: points[0], phi_param: points[1]}) 
         for points in coords[index*75:(index+1)*75]]
        qobj = assemble(transpile(circuits, backend=backend), backend=backend, shots=N_shots)
        runningjobs.append(backend.run(qobj))
        job_ids.append(runningjobs[-1].job_id())
        with open('job_ids_santiago.txt', 'a') as file:
            file.write(runningjobs[-1].job_id()+"\n")
        if(backend.job_limit().active_jobs == backend.job_limit().maximum_jobs):
            print("Maximum number of jobs reached. Waiting...")
            try:
                job_result = runningjobs[0].result()  # It will block until the job finishes.
                print("The job finished with result {}".format(job_result))
                runningjobs.pop(0)
            except JobError as ex:
                print("Something wrong happened!: {}".format(ex))
    else:
        circuits = [circuit.bind_parameters(
        {theta_param: points[0], phi_param: points[1]}) 
         for points in coords[index*75:]]
        qobj = assemble(transpile(circuits, backend=backend), backend=backend, shots=N_shots)
        runningjobs.append(backend.run(qobj))
        job_ids.append(runningjobs[-1].job_id())
        with open('job_ids_santiago.txt', 'a') as file:
            file.write(runningjobs[-1].job_id()+"\n")
        if(backend.job_limit().active_jobs == backend.job_limit().maximum_jobs):
            print("Maximum number of jobs reached. Waiting...")
            try:
                job_result = runningjobs[0].result()  # It will block until the job finishes.
                print("The job finished with result {}".format(job_result))
                runningjobs.pop(0)
            except JobError as ex:
                print("Something wrong happened!: {}".format(ex))
    index=index+1

In [ ]:
read_job_ids = open("job_ids_santiago.txt").read().splitlines()
counts_array=[]
results_probabilities=[]
retrieved_jobs=[backend.retrieve_job(jobid) for jobid in read_job_ids]
for job in retrieved_jobs:
    counts_array=counts_array+job.result().get_counts()
for job in counts_array:
    results_probabilities.append((job["000"]/N_shots,job["001"]/N_shots))
results_probabilities=np.array(results_probabilities)

In [ ]:
data_sheet=pd.DataFrame(data=np.hstack((target_points,results_probabilities)),columns=["θ","ϕ","prob_0","prob_1"])

In [ ]:
data_sheet.head()

In [ ]:
for (point,probabilities) in zip(target_points,results_probabilities):
    with open('results_ibm_santiago.txt', 'a') as file:
        file.write(str(point[0])+"\t"+str(point[1])+"\t"+str(probabilities[0])+"\t"+str(probabilities[1])+"\n")

In [ ]:
data_sheet=pd.DataFrame(data=np.hstack((target_points,results_probabilities)),columns=["θ","ϕ","prob_0","prob_1"])

In [ ]:
data_sheet.head()

In [ ]:
# save to excel, optional
data_sheet.to_excel("data_sheet_ibm_santiago.xlsx")

In [ ]:
data_sheet=pd.read_excel("data_sheet_ibm_santiago.xlsx")

In [ ]:
# Make data.
thetas=data_sheet["θ"]
phis=data_sheet["ϕ"]
x, y, z = np.cos(phis) * np.sin(thetas), np.sin(phis) * np.sin(thetas), np.cos(thetas)
# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0/(5/6), template="plotly_white")

fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(data_sheet.prob_0))
print("Standard deviation: ", np.std(data_sheet.prob_0))
